In [1]:
# Configuration - Set Your Artist and Username
ARTIST_NAME = "Stone Temple Pilots"  # Change this to any artist
USERNAME = "sugarsmax"               # Change this to your Last.fm username

print(f"🎵 Last.fm Track Comparison")
print(f"Artist: {ARTIST_NAME}")
print(f"Username: {USERNAME}")
print(f"Ready to scrape and compare!")


🎵 Last.fm Track Comparison
Artist: Stone Temple Pilots
Username: sugarsmax
Ready to scrape and compare!


In [2]:
# Scrape Personal Top Tracks
from urllib.parse import quote_plus

# Navigate to your personal tracks page
personal_url = f"https://www.last.fm/user/{USERNAME}/library/music/{quote_plus(ARTIST_NAME)}/+tracks"
print(f"📊 Scraping your personal tracks from: {personal_url}")

mcp_playwright_browser_navigate({"url": personal_url})

# Extract personal track data
personal_js = """
() => {
    const tracks = [];
    const rows = document.querySelectorAll('table tbody tr');
    
    rows.forEach((row, index) => {
        const cells = row.querySelectorAll('td');
        if (cells.length >= 8 && !row.textContent.includes('Want your own')) {
            const rank = parseInt(cells[0].textContent.trim());
            const name = cells[3].querySelector('a')?.textContent.trim();
            const scrobbles = parseInt(cells[6].textContent.trim());
            
            if (rank && name && scrobbles) {
                tracks.push({
                    rank: rank,
                    name: name,
                    scrobbles: scrobbles
                });
            }
        }
    });
    
    return tracks;
}
"""

personal_tracks = mcp_playwright_browser_evaluate({"function": personal_js})
print(f"✅ Found {len(personal_tracks)} personal tracks")

# Show first few tracks
for track in personal_tracks[:5]:
    print(f"  #{track['rank']}: {track['name']} ({track['scrobbles']} scrobbles)")


📊 Scraping your personal tracks from: https://www.last.fm/user/sugarsmax/library/music/Stone+Temple+Pilots/+tracks


NameError: name 'mcp_playwright_browser_navigate' is not defined

In [ ]:
# Scrape Global Top Tracks

# Navigate to global tracks page
global_url = f"https://www.last.fm/music/{quote_plus(ARTIST_NAME)}/+tracks"
print(f"🌍 Scraping global tracks from: {global_url}")

mcp_playwright_browser_navigate({"url": global_url})

# Extract global track data
global_js = """
() => {
    const tracks = [];
    const rows = document.querySelectorAll('table tbody tr');
    
    rows.forEach((row, index) => {
        const cells = row.querySelectorAll('td');
        if (cells.length >= 8) {
            const rank = parseInt(cells[0].textContent.trim());
            const name = cells[3].querySelector('a')?.textContent.trim();
            const scrobbles = parseInt(cells[6].textContent.replace(/,/g, '').trim());
            
            if (rank && name && scrobbles) {
                tracks.push({
                    rank: rank,
                    name: name,
                    scrobbles: scrobbles
                });
            }
        }
    });
    
    return tracks;
}
"""

global_tracks = mcp_playwright_browser_evaluate({"function": global_js})
print(f"✅ Found {len(global_tracks)} global tracks")

# Show first few tracks
for track in global_tracks[:5]:
    print(f"  #{track['rank']}: {track['name']} ({track['scrobbles']:,} listeners)")


In [ ]:
# Generate Simple Comparison Table
from datetime import datetime

# Normalize track names for matching
def normalize_name(name):
    return name.replace(" - 2017 Remaster", "").replace(" - 2019 Remaster", "").replace(" - Acoustic", "").strip()

# Create global lookup
global_lookup = {}
for track in global_tracks:
    normalized = normalize_name(track["name"])
    if normalized not in global_lookup:
        global_lookup[normalized] = track

# Build comparison table
print(f"# {ARTIST_NAME} - Personal vs Global Rankings")
print(f"")
print(f"**Username:** {USERNAME}")
print(f"**Date:** {datetime.now().strftime('%Y-%m-%d')}")
print(f"")
print(f"| Personal Rank | Track Name | Global Rank | Difference | Scrobbles |")
print(f"|---------------|------------|-------------|------------|-----------|")

found_count = 0
missing_count = 0

for p_track in personal_tracks:
    p_normalized = normalize_name(p_track["name"])
    
    if p_normalized in global_lookup:
        g_track = global_lookup[p_normalized]
        diff = p_track["rank"] - g_track["rank"]
        diff_str = f"+{diff}" if diff > 0 else str(diff)
        global_rank = g_track["rank"]
        found_count += 1
    else:
        global_rank = "Not in Top 20"
        diff_str = "N/A"
        missing_count += 1
    
    print(f"| #{p_track['rank']} | {p_track['name']} | {global_rank} | {diff_str} | {p_track['scrobbles']} |")

print(f"")
print(f"**Summary:**")
print(f"- Total personal tracks: {len(personal_tracks)}")
print(f"- Found in global top 20: {found_count}")
print(f"- Missing from global (deep cuts): {missing_count}")
print(f"- Deep cut percentage: {(missing_count/len(personal_tracks)*100):.1f}%")

# Save to file
safe_artist = ARTIST_NAME.lower().replace(" ", "_").replace("&", "and")
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"lastfm_comparison_{safe_artist}_{timestamp}.md"

markdown_content = f"""# {ARTIST_NAME} - Personal vs Global Rankings

**Username:** {USERNAME}
**Date:** {datetime.now().strftime('%Y-%m-%d')}

| Personal Rank | Track Name | Global Rank | Difference | Scrobbles |
|---------------|------------|-------------|------------|-----------|
"""

for p_track in personal_tracks:
    p_normalized = normalize_name(p_track["name"])
    if p_normalized in global_lookup:
        g_track = global_lookup[p_normalized]
        diff = p_track["rank"] - g_track["rank"]
        diff_str = f"+{diff}" if diff > 0 else str(diff)
        global_rank = g_track["rank"]
    else:
        global_rank = "Not in Top 20"
        diff_str = "N/A"
    
    markdown_content += f"| #{p_track['rank']} | {p_track['name']} | {global_rank} | {diff_str} | {p_track['scrobbles']} |\n"

markdown_content += f"""
**Summary:**
- Total personal tracks: {len(personal_tracks)}
- Found in global top 20: {found_count}
- Missing from global (deep cuts): {missing_count}
- Deep cut percentage: {(missing_count/len(personal_tracks)*100):.1f}%
"""

with open(filename, 'w', encoding='utf-8') as f:
    f.write(markdown_content)

print(f"")
print(f"💾 Report saved to: {filename}")
print(f"🎵 Analysis complete for {ARTIST_NAME}!")
